# Time Series — Univariate Forecast with Exogenous (US Retail Sales w/ CPI)


# Pinned installs for reproducibility in Colab
!pip -q install "pycaret>=3.0.4,<4" "pandas-datareader>=0.10.0" xgboost lightgbm catboost --upgrade

import os, sys, platform, subprocess, pandas as pd, numpy as np
import matplotlib.pyplot as plt

# Show GPU status (Colab: Runtime -> Change runtime type -> GPU)
try:
    import subprocess
    print(subprocess.check_output(["nvidia-smi"]).decode("utf-8"))
except Exception as e:
    print("No NVIDIA GPU detected (this is OK; PyCaret will fall back to CPU).")

print("PyCaret version check:")
import pycaret
import sklearn
print("pycaret", pycaret.__version__, "| sklearn", sklearn.__version__)


In [ ]:

import pandas as pd
from pandas_datareader import data as pdr

# Pull monthly Retail Sales (not SA) and CPI from FRED
retail = pdr.DataReader("RSXFSN", "fred", start="2000-01-01")  # target
cpi = pdr.DataReader("CPIAUCSL", "fred", start="2000-01-01")

# Align and fill
df = retail.join(cpi, how="inner").dropna()
y = df["RSXFSN"].asfreq("MS").interpolate().rename("retail_sales")
X = pd.DataFrame({"CPIAUCSL": df["CPIAUCSL"].asfreq("MS").interpolate()})

from pycaret.time_series import TSForecastingExperiment
exp = TSForecastingExperiment()
exp.setup(data=y, exogenous=X, fh=12, fold=3, session_id=42)

best = exp.compare_models()
final = exp.finalize_model(best)
future = exp.predict_model(final)
exp.plot_model(final, plot="forecast")
future.head()
